In [1]:
import DigitalContactTracing as DCT
from load_temporal_graph import *

%load_ext autoreload
%autoreload 2

In [2]:

PARAMETERS = {"temporal_gap" : 900,
              "memory_contacts" : 7, # days
              "max_time_quar" : 10, # days
              "policies" : {"policy1":[-100,600]},
              "eps_Is" : [0.5,0.2],
              "times" : 2,
              "seed" : 11,
              "Y_i" : 10,
              "nc" : 0.2,
              "beta_t": 0.002,
              "symptomatics" : 0.8,   # number of symptomatic people (old notation: p)
              "testing" : 0.25,
              "store":{"to_store": True,
                       "path_to_store":"RESULTS/"}} 


In [3]:
df = load_df("Dataset//bt_symmetric.csv",n_row =100000)#,n_individuals=100)
graphs = build_graphs(get_array_of_contacts(df,PARAMETERS["temporal_gap"]),PARAMETERS["temporal_gap"])

In [6]:

np.random.seed(PARAMETERS["seed"])

for eps_I in PARAMETERS["eps_Is"]:
    for policy in PARAMETERS["policies"]:
        filter_rssi = PARAMETERS["policies"][policy][0]
        filter_duration = PARAMETERS["policies"][policy][1]
        results = []
        for i in range(PARAMETERS["times"]):
            print("-"*100)
            dct = DCT.DigitalContactTracing(graphs = graphs,
                                            PARAMETERS = PARAMETERS,
                                            eps_I=eps_I,
                                            filter_rssi=filter_rssi,
                                            filter_duration=filter_duration)
            
            res = dct.simulate()
            results.append(res)
            
        DCT.store_results(results,PARAMETERS,filter_rssi,filter_duration,eps_I)
        
        

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Simulation saved in  RESULTS/
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Simulation saved in  RESULTS/
